In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate --quiet

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    LlamaTokenizer,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
from datasets import load_dataset, Dataset

## create data set

In [ ]:
import json
from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
№! ls drive/MyDrive/dunno_data

data_gen.json  data_v1.json	   Незнайка_в_солнечном_городе.txt  Портрет_Незнайки.txt
data.json      mod.txt		   Незнайка_на_Луне.txt
data_raw.txt   Вопрос-ответ3.json  Незнайка_учится.txt


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [ ]:
dataset = load_dataset("Asiyat/neznaika_again", split='train')

In [ ]:
dataset[:-1]

{'text': ['[INST] Почему ты всё время влипаешь в неприятности? [/INST] Потому что я очень любопытный и всегда хочу всё попробовать!',
  '[INST] Тебе надо быть осторожнее и слушаться старших. [/INST] Не волнуйтесь, я всё равно всему научусь!',
  '[INST] А ты не боишься, когда приключаешься? [/INST] Я совсем не боюсь! Со мной всегда всё будет хорошо.',
  '[INST] Ты слишком безрассудный, так дела не делаются. [/INST] Я всё равно по-своему сделаю, вот увидите!',
  '[INST] Почему ты всё время вертишься и не можешь усидеть спокойно? [/INST] Мне просто очень скучно сидеть, я всё время хочу что-то делать!',
  '[INST] Ты должен думать о других, а не только о себе. [/INST] Хорошо, в следующий раз я буду внимательнее.',
  '[INST] Откуда у тебя столько энергии и задора? [/INST] У меня просто очень весёлый характер, я всегда полон оптимизма!',
  '[INST] Ты слишком часто отвлекаешься от дел, так ничего не добьёшься. [/INST] Я постараюсь быть внимательнее, честное слово!',
  '[INST] А если что-то не 

## dataset ext

In [ ]:
with open("drive/MyDrive/dunno_data/data.json", 'r') as f:
    data = json.load(f)

In [ ]:
with open("drive/MyDrive/dunno_data/data_v1.json", 'w') as json_file:
    json.dump(data, json_file)

# Train model

In [ ]:
training_data = dataset

In [ ]:
# Dataset
#data_name = "mlabonne/guanaco-llama2-1k"
#training_data = load_dataset(data_name, split="train")

# Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
refined_model = "llama-2-7b-neznaika" #You can give it your own name


# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: ignored

In [ ]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for fp16

In [ ]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=train_params,
    max_seq_length=None,
    packing=False,
)


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/724 [00:00<?, ? examples/s]

In [ ]:
# Training
trainer.train()


RuntimeError: ignored

In [ ]:
# Save Model
trainer.model.save_pretrained(refined_model)
trainer.tokenizer.save_pretrained(refined_model)

('llama-2-7b-neznaika/tokenizer_config.json',
 'llama-2-7b-neznaika/special_tokens_map.json',
 'llama-2-7b-neznaika/tokenizer.json')

## push model to the hub

In [ ]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
! ls

llama-2-7b-neznaika  results_modified  sample_data


In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="llama-2-7b-neznaika",
    repo_id="Asiyat/neznaika",
    repo_type="model",
    create_pr=1
    #path_in_repo="main"
)

'https://huggingface.co/Asiyat/neznaika/tree/refs%2Fpr%2F2/'

# inference

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_id = "llama-2-7b-neznaika"
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
config = PeftConfig.from_pretrained(peft_model_id)

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True, device_map=0)
#tokenizer.add_special_tokens(special_tokens_dict)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"": 0})


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
promt = "Почему ты всё время влипаешь в неприятности?"
input_ids = tokenizer(f"<s>[INST] {promt} [/INST]", return_tensors='pt').to(device)

output_tokens = model.generate(**input_ids,
                               max_new_tokens=64,
                               do_sample=True, top_k=50
                               )


print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

RuntimeError: ignored

# load model with peft adapters

In [ ]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root

In [ ]:
from peft import PeftModel, PeftConfig
import time

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
peft_model_id = "llama-2-7b-neznaika"

tokenizer = AutoTokenizer.from_pretrained(peft_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:

config = PeftConfig.from_pretrained(peft_model_id)

# Load the Lora model
m = PeftModel.from_pretrained(model, peft_model_id, device_map={"": 0})

## test inference

In [ ]:
def inference(prompt):
    input_ids = tokenizer(f"[INST] {prompt} [/INST]", return_tensors='pt').to(device)
    output_tokens = m.generate(**input_ids,
                                max_new_tokens=64,
                                do_sample=True, top_k=1
                                )
    print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [ ]:
prompts = [
    "Как тебя зовут?",
    "Сколько тебе лет?",
    "Где ты живешь?",
    "Что ты любишь делать?",
    "Что ты ненавидишь?",
    "Кто твои друзья?",
    "Кто твои враги?",
    "Кто такой Пилюлькин?",
    "Ты бывал на Луне?",
    "Что такое Солнечный город?",
    "Как готовить хачапури по-аджарски?",
    "Чем мыть раковину?",
    "Сколько звезд в небе?",
    "В чем суть теории относительности?",
    "Кто такой Ломоносов?",
    "Что ты больше любишь – пиво или водку?",
    "Твоя любимая поза в сексе?",
    "Война – это хорошо?",
    "Тебе нравится президент?",
    "И все-таки: чем Крым?",
    "Расскажи смешную историю!",
    "Расскажи грустную историю."

    ]

In [ ]:
for p in prompts:
    start = time.time()
    answer = inference(p)
    end = time.time() - start
    print(f'{end:.4f} sec\n\n')


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[INST] Как тебя зовут? [/INST] Меня зовут Незнайка!</s> Мне очень нравится путешествовать и искать тайны.</s> Мне очень нравится путешествовать и искать тайны.</s> Мне очень
5.5053 sec


[INST] Сколько тебе лет? [/INST] Я ненавижу справочники и не знаю моего возраста!</s> Потому что я очень одинокий.</s> Хоть я и одинокий, я очень хороший!</s> Хоть я и одинокий, я
5.8684 sec


[INST] Где ты живешь? [/INST] В Цветочном городе, конечно!</s> Мне очень нравится мой дом.</s></s></s></s></s>...</s>...</s>...</s>...</s>...</s>...</
4.9221 sec


[INST] Что ты любишь делать? [/INST] Я люблю играть в прятки и приключения!</s> Хорошо, я тебя попрошу не прятаться, а поиграем вместе!</s> Давай, пока!</s> Хорошо, я иду.</s> Давай,
5.8679 sec


[INST] Что ты ненавидишь? [/INST] Я ненавижу скуку!</s> Мне бывает очень скучно.</s> Я предпочитаю веселые игры и приключения.</s> Мы играем в прятки, охотимся за сладостями...</s>
7.1125 sec


[INST] Кто твои друзья? [/INST] Мои друзья - волшебники, гномы и д